In [1]:
import pathlib
import tempfile
from urllib.parse import urlparse
from urllib.request import urlretrieve

import numpy as np
import asdf
from astropy.io import fits
from astropy.modeling import models, fitting
import astropy.units as u

from gwcs import wcs as gwcs
from gwcs import coordinate_frames as cf

from grism_observation import GrismObs
from HST.hst_grism_reffiles import create_tsgrism_wavelengthrange, create_grism_specwcs
from HST.generate_wfc3_distortion import create_wfc3_distortion

**WARNING**: LOCAL JWST PRD VERSION PRDOPSSOC-034 DOESN'T MATCH THE CURRENT ONLINE VERSION PRDOPSSOC-036
Please consider updating pysiaf, e.g. pip install --upgrade pysiaf or conda update pysiaf


In [ ]:
reference_files = dict()

conf_filepath = "HST/UVIS_G280_CCD1_V2.conf"

filter = "G280"

specwcs_filename = "WFC3_" + str(filter) + "_specwcs.asdf"
create_grism_specwcs(conffile=str(conf_filepath), pupil=filter, outname=specwcs_filename)

#### Create inverse SIP coefficients for UVIS

In [ ]:
sip_file = "/Users/rosteen/Data/UVIS/G280/icwz15e7q_flt.fits"
sip_hdus = fits.open(str(sip_file))

# CCD 1
acoef_1 = dict(sip_hdus[1].header['A_*'])
a_order_1 = acoef_1.pop('A_ORDER')
bcoef_1 = dict(sip_hdus[1].header['B_*'])
b_order_1 = bcoef_1.pop('B_ORDER')

# CCD 2
acoef_2 = dict(sip_hdus[4].header['A_*'])
a_order_2 = acoef_2.pop('A_ORDER')
bcoef_2 = dict(sip_hdus[4].header['B_*'])
b_order_2 = bcoef_2.pop('B_ORDER')

In [ ]:
acoef_1

In [ ]:
acoef_2

In [ ]:
sip_hdus[1].data.shape

In [ ]:
indices = np.indices((2051, 4096))
print(indices)

In [ ]:
indices.shape

In [ ]:
x_indices = indices[0,:]
y_indices = indices[1,:]

In [ ]:
a_polycoef_1 = {}
for key in acoef_1:
    a_polycoef_1['c' + key.split('A_')[1]] = acoef_1[key]

b_polycoef_1 = {}
for key in bcoef_1:
    b_polycoef_1['c' + key.split('B_')[1]] = bcoef_1[key]
    
a_polycoef_2 = {}
for key in acoef_2:
    a_polycoef_2['c' + key.split('A_')[1]] = acoef_2[key]

b_polycoef_2 = {}
for key in bcoef_2:
    b_polycoef_2['c' + key.split('B_')[1]] = bcoef_2[key]

In [ ]:
a_poly_1 = models.Polynomial2D(a_order_1, **a_polycoef_1)
b_poly_1 = models.Polynomial2D(b_order_1, **b_polycoef_1)

a_poly_2 = models.Polynomial2D(a_order_2, **a_polycoef_2)
b_poly_2 = models.Polynomial2D(b_order_2, **b_polycoef_2)

In [ ]:
result_x_1 = a_poly_1(x_indices, y_indices)
result_y_1 = b_poly_1(x_indices, y_indices)

result_x_2 = a_poly_2(x_indices, y_indices)
result_y_2 = b_poly_2(x_indices, y_indices)

In [ ]:
result_x_1

In [ ]:
x_indices + result_x_1

In [ ]:
# Initialize a 5th degree 2D polynomial for fitting
inverse_model = models.Polynomial2D(5)

In [ ]:
inverse_model

In [ ]:
fitter = fitting.LevMarLSQFitter()

best_fit = fitter(inverse_model, x_indices+result_x_1, y_indices+result_y_1, -result_x_1)
x_inverse_model = best_fit

best_fit = fitter(inverse_model, x_indices+result_x_1, y_indices+result_y_1, -result_y_1)
y_inverse_model = best_fit

In [ ]:
x_inverse_model

In [ ]:
y_inverse_model

In [ ]:
x_inverse_model(x_indices+result_x_1, y_indices+result_y_1)

In [ ]:
(x_inverse_model(x_indices+result_x_1, y_indices+result_y_1) + result_x_1).max()

In [ ]:
(y_inverse_model(x_indices+result_x_1, y_indices+result_y_1) + result_y_1).max()

In [ ]:
fitter = fitting.LevMarLSQFitter()

best_fit = fitter(inverse_model, x_indices+result_x_2, y_indices+result_y_2, -result_x_2)
x_inverse_model_2 = best_fit

best_fit = fitter(inverse_model, x_indices+result_x_2, y_indices+result_y_2, -result_y_2)
y_inverse_model_2 = best_fit

In [ ]:
# Testing to see if this actually worked...

crpix = [sip_hdus[1].header['CRPIX1'], sip_hdus[1].header['CRPIX2']]

crval = [sip_hdus[1].header['CRVAL1'],
         sip_hdus[1].header['CRVAL2']]

cdmat = np.array([[sip_hdus[1].header['CD1_1'], sip_hdus[1].header['CD1_2']],
                  [sip_hdus[1].header['CD2_1'], sip_hdus[1].header['CD2_2']]])

SIP_forward = (models.Shift(-(crpix[0]-1)) & models.Shift(-(crpix[1]-1)) | # Calculate u and v
              models.Mapping((0, 1, 0, 1, 0, 1)) | a_poly_1 & b_poly_1 & models.Identity(2) |
              models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
              models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() |
              models.RotateNative2Celestial(crval[0], crval[1], 180))

SIP_backward = (models.RotateCelestial2Native(crval[0], crval[1], 180) |
             models.Sky2Pix_TAN() | models.AffineTransformation2D(matrix=cdmat).inverse |
             models.Mapping((0, 1, 0, 1, 0, 1)) | x_inverse_model & y_inverse_model & models.Identity(2) |
             models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
             models.Shift((crpix[0]-1)) & models.Shift((crpix[1]-1)))

full_distortion_model = SIP_forward
full_distortion_model.inverse = SIP_backward

imagepipe = []

det_frame = cf.Frame2D(name="detector")
imagepipe.append((det_frame, full_distortion_model))
world_frame = cf.CelestialFrame(name="world", unit = (u.Unit("deg"), u.Unit("deg")),
                                axes_names=('lon', 'lat'), axes_order=(0, 1),
                                reference_frame="ICRS")
imagepipe.append((world_frame, None))
geo_transforms = gwcs.WCS(imagepipe)

In [ ]:
geo_transforms.available_frames

In [ ]:
forward = geo_transforms.get_transform("detector", "world")

In [ ]:
backward = geo_transforms.get_transform("world", "detector")

In [ ]:
forward(2048, 1026)

In [ ]:
backward(206.43128029664106, 26.4186090137141)

In [ ]:
# Now test the models for CCD2

crpix = [sip_hdus[4].header['CRPIX1'], sip_hdus[4].header['CRPIX2']]

crval = [sip_hdus[4].header['CRVAL1'],
         sip_hdus[4].header['CRVAL2']]

cdmat = np.array([[sip_hdus[4].header['CD1_1'], sip_hdus[4].header['CD1_2']],
                  [sip_hdus[4].header['CD2_1'], sip_hdus[4].header['CD2_2']]])

SIP_forward = (models.Shift(-(crpix[0]-1)) & models.Shift(-(crpix[1]-1)) | # Calculate u and v
              models.Mapping((0, 1, 0, 1, 0, 1)) | a_poly_2 & b_poly_2 & models.Identity(2) |
              models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
              models.AffineTransformation2D(matrix=cdmat) | models.Pix2Sky_TAN() |
              models.RotateNative2Celestial(crval[0], crval[1], 180))

SIP_backward = (models.RotateCelestial2Native(crval[0], crval[1], 180) |
             models.Sky2Pix_TAN() | models.AffineTransformation2D(matrix=cdmat).inverse |
             models.Mapping((0, 1, 0, 1, 0, 1)) | x_inverse_model_2 & y_inverse_model_2 & models.Identity(2) |
             models.Mapping((0, 2, 1, 3)) | models.math.AddUfunc() & models.math.AddUfunc() |
             models.Shift((crpix[0]-1)) & models.Shift((crpix[1]-1)))

full_distortion_model = SIP_forward
full_distortion_model.inverse = SIP_backward

imagepipe = []

det_frame = cf.Frame2D(name="detector")
imagepipe.append((det_frame, full_distortion_model))
world_frame = cf.CelestialFrame(name="world", unit = (u.Unit("deg"), u.Unit("deg")),
                                axes_names=('lon', 'lat'), axes_order=(0, 1),
                                reference_frame="ICRS")
imagepipe.append((world_frame, None))
geo_transforms = gwcs.WCS(imagepipe)

In [ ]:
forward = geo_transforms.get_transform("detector", "world")
backward = geo_transforms.get_transform("world", "detector")

In [ ]:
forward(2048, 1026)

In [ ]:
backward(206.4568554451976, 26.418285311669543)

In [ ]:
x_inverse_model_2

In [ ]:
y_inverse_model_2

In [ ]:
y_inverse_model_2.parameters

In [ ]:
y_inverse_model_2.param_names

In [ ]:
getattr(y_inverse_model_2, "c0_0").value

In [ ]:
# Loop through the models to get the inverse coefficients AP_* and BP_*
from datetime import datetime
ir_sip = fits.open("config/HST/WFC3_IR_distortion.fits")

hdus = fits.HDUList()
hdus.append(ir_sip[0])
hdus.append(fits.ImageHDU(header=fits.Header(), data=None))
hdus.append(fits.ImageHDU(header=fits.Header(), data=None))

hdr = hdus[0].header
hdr["MADEBY"] = "Ricky O'Steen"
hdr["DETECTOR"] = "UVIS"
hdr["MADEON"] = str(datetime.now())
hdr["NCHIP"] = 2
hdus[0].header = hdr

In [ ]:
hdus[0].header

In [ ]:
# CCD 2 (comes first, in HDU 1 for some reason)

hdr = hdus[1].header
hdr["EXTNAME"] = "UVIS"
hdr["CHIPNAME"] = "UVIS"
hdr["CCDCHIP"] = sip_hdus[1].header["CCDCHIP"]

hdr["A_ORDER"] = a_order_1
for key in acoef_1:
    hdr[key] = acoef_1[key]
    
hdr["B_ORDER"] = b_order_1
for key in bcoef_1:
    hdr[key] = bcoef_1[key]
    
hdr["AP_ORDER"] = 5
for param in x_inverse_model.param_names:
    key = "AP_{}_{}".format(param[1], param[3])
    hdr[key] = getattr(x_inverse_model, param).value
    
hdr["BP_ORDER"] = 5
for param in y_inverse_model.param_names:
    key = "BP_{}_{}".format(param[1], param[3])
    hdr[key] = getattr(y_inverse_model, param).value

hdus[1].header = hdr

# CCD 1 (they're in reverse order in the original file)

hdr = hdus[2].header
hdr["EXTNAME"] = "UVIS"
hdr["CHIPNAME"] = "UVIS"
hdr["CCDCHIP"] = sip_hdus[4].header["CCDCHIP"]

hdr["A_ORDER"] = a_order_2
for key in acoef_2:
    hdr[key] = acoef_2[key]
    
hdr["B_ORDER"] = b_order_2
for key in bcoef_2:
    hdr[key] = bcoef_2[key]
    
hdr["AP_ORDER"] = 5
for param in x_inverse_model_2.param_names:
    key = "AP_{}_{}".format(param[1], param[3])
    hdr[key] = getattr(x_inverse_model_2, param).value
    
hdr["BP_ORDER"] = 5
for param in y_inverse_model_2.param_names:
    key = "BP_{}_{}".format(param[1], param[3])
    hdr[key] = getattr(y_inverse_model_2, param).value

hdus[2].header = hdr

In [ ]:
hdus[1].header

In [ ]:
hdus[2].header

In [ ]:
hdus.writeto("config/HST/WFC3_UVIS_distortion.fits", overwrite=True)

#### Testing backward and forward dispersion to confirm round trip

In [2]:
# Try out a GrismObs for UVIS with no INVDISPY defined to see if we can get anything

sip_file = "/Users/rosteen/Data/UVIS/G280/icwz15e7q_flt.fits"

test = GrismObs(sip_file)

In [ ]:
import asdf
from HST.dispersion_models import DISPXY_Extension

asdf.get_config().add_extension(DISPXY_Extension())

In [ ]:
specwcs = asdf.open("config/HST/WFC3_G280_specwcs.asdf").tree
print(specwcs.keys())

In [ ]:
temp_model = specwcs['invdispl'][1]
temp_model

In [ ]:
temp_model.n_inputs

In [3]:
test.geometric_transforms.available_frames

['grism_detector', 'detector', 'world']

In [4]:
g2d = test.geometric_transforms.get_transform("grism_detector", "detector")

In [5]:
d2g = test.geometric_transforms.get_transform("detector", "grism_detector")

In [6]:
g2d

<WFC3IRForwardGrismDispersion(name='wfc3ir_forward_row_grism_dispersion')>

In [7]:
g2d(100,100,105,105,1)

AttributeError: 'DISPXY_Model' object has no attribute '_model_set_axis'

In [8]:
d2g(100,100,150, 1)

Error in lmodel evaluation
N inputs: 3


AttributeError: 'DISPXY_Model' object has no attribute '_model_set_axis'

In [ ]:
g2d.has_inverse()

In [ ]:
d2g.has_inverse()